<p><a name="sections"></a></p>


# Sections

- <a href="#DS">Data Loading</a><br>
- <a href="#DC">Data Cleaning</a><br>
- <a href="#DM">Data Manipulation</a><br>


In [430]:
import numpy as np
import pandas as pd
import re
import quandl
import datetime

<p><a name="DL"></a></p>
### Data Loading

- Load the Data from the scrapy output GlassDoor2.csv

In [716]:
ReviewsRaw = pd.read_csv('GlassDoor2.csv')

- Load the Data from the scrapy Overview output - this contains the metadata from top 100 list

In [ ]:
CompOverview = pd.read_csv('GlassDoorOverview.csv')

In [406]:
CompOverview.head()

,CompanyHQ,Industry,Name,Revenue,Size,Type
0,"Menlo Park, CA",Internet,Facebook,$5 to $10 billion (USD) per year,10000+ employees,Company - Public (FB)
1,"Boston, MA",Consulting,Bain & Company,$2 to $5 billion (USD) per year,5001 to 10000 employees,Company - Private
2,"Boston, MA",Consulting,Boston Consulting Group,$2 to $5 billion (USD) per year,Unknown,Company - Private
3,"Irvine, CA",Fast-Food & Quick-Service Restaurants,In-N-Out Burger,$100 to $500 million (USD) per year,10000+ employees,Company - Private
4,"Mountain View, CA",Internet,Google,$10+ billion (USD) per year,10000+ employees,Company - Public (GOOG)


In [667]:
ReviewsRaw.Name.head()[1]

' SAP Concur '

### Stock Data
- Retrieve stock data for all publicly traded companies from list

#### Quandl
- Quandl - able to successfully get only 40 or so of the 49 companies. 7 missing, 2 have surprisingly short time series data.

In [24]:
quandl.ApiConfig.api_key = "QxQBHqhaqAhrtcGJVirJ"

In [77]:
Public = pd.DataFrame({"Public": CompOverview.Type.str.contains('public',case=False)})
Ticker = pd.DataFrame({"Ticker": list(map(lambda s: s[s.find("(")+1:s.find(")")],CompOverview.Type))})

Overview2 = pd.concat((CompOverview,Public,Ticker),axis=1)
Overview2.Ticker = Overview2.Ticker*Overview2.Public


In [427]:
OverviewPublic = Overview2[Overview2.Ticker!='']
OverviewPublic = OverviewPublic[OverviewPublic.index != 42] #drop Protiviti - not actually public/traded
OverviewPublic.index = range(0,len(OverviewPublic))
TickerList = OverviewPublic.Ticker

In [698]:
CompOverview

,CompanyHQ,Industry,Name,Revenue,Size,Type
0,"Menlo Park, CA",Internet,Facebook,$5 to $10 billion (USD) per year,10000+ employees,Company - Public (FB)
1,"Boston, MA",Consulting,Bain & Company,$2 to $5 billion (USD) per year,5001 to 10000 employees,Company - Private
2,"Boston, MA",Consulting,Boston Consulting Group,$2 to $5 billion (USD) per year,Unknown,Company - Private
3,"Irvine, CA",Fast-Food & Quick-Service Restaurants,In-N-Out Burger,$100 to $500 million (USD) per year,10000+ employees,Company - Private
4,"Mountain View, CA",Internet,Google,$10+ billion (USD) per year,10000+ employees,Company - Public (GOOG)
5,"Vancouver, BC (Canada)","Department, Clothing, & Shoe Stores",lululemon,$2 to $5 billion (USD) per year,1001 to 5000 employees,Company - Public (LULU)
6,"Cambridge, MA",Computer Hardware & Software,HubSpot,$100 to $500 million (USD) per year,1001 to 5000 employees,Company - Public (HUBS)
7,"Maryland Heights, MO",Computer Hardware & Software,World Wide Technology,$5 to $10 billion (USD) per year,1001 to 5000 employees,Company - Private
8,"Memphis, TN",NaN,St. Jude Children's Research Hospital,$100 to $500 million (USD) per year,1001 to 5000 employees,Nonprofit Organization
9,"Weston, FL",Enterprise Software & Network Solutions,Ultimate Software,$500 million to $1 billion (USD) per year,1001 to 5000 employees,Company - Public (ULTI)


In [704]:
OverviewPublic

,CompanyHQ,Industry,Name,Revenue,Size,Type,Public,Ticker
0,"Menlo Park, CA",Internet,Facebook,$5 to $10 billion (USD) per year,10000+ employees,Company - Public (FB),True,FB
1,"Mountain View, CA",Internet,Google,$10+ billion (USD) per year,10000+ employees,Company - Public (GOOG),True,GOOG
2,"Vancouver, BC (Canada)","Department, Clothing, & Shoe Stores",lululemon,$2 to $5 billion (USD) per year,1001 to 5000 employees,Company - Public (LULU),True,LULU
3,"Cambridge, MA",Computer Hardware & Software,HubSpot,$100 to $500 million (USD) per year,1001 to 5000 employees,Company - Public (HUBS),True,HUBS
4,"Weston, FL",Enterprise Software & Network Solutions,Ultimate Software,$500 million to $1 billion (USD) per year,1001 to 5000 employees,Company - Public (ULTI),True,ULTI
5,"Walldorf, Baden-Württemberg (Germany)",Computer Hardware & Software,SAP,$10+ billion (USD) per year,10000+ employees,Company - Public (SAP),True,SAP
6,"San Francisco, CA",Computer Hardware & Software,Salesforce,$5 to $10 billion (USD) per year,10000+ employees,Company - Public (CRM),True,CRM
7,"Atlanta, GA",Airlines,Delta Air Lines,$10+ billion (USD) per year,10000+ employees,Company - Public (DAL),True,DAL
8,"Dallas, TX",Airlines,Southwest Airlines,$10+ billion (USD) per year,10000+ employees,Company - Public (LUV),True,LUV
9,"Santa Clara, CA",Computer Hardware & Software,NVIDIA,$5 to $10 billion (USD) per year,10000+ employees,Company - Public (NVDA),True,NVDA


- <b>OverviewPublic</b> now contains the overview data for 49 companies of the original review list. 
- TickerList contains a list of the stock tickers for these companies only

#### Quandl API function

In [198]:
def quandl_stocks(symbol, start_date=(2010, 1, 1), end_date=None):
    query_list = ['WIKI' + '/' + symbol + '.' + str(k) for k in [1,11]]
 
    start_date = datetime.date(*start_date)
 
    if end_date:
        end_date = datetime.date(*end_date)
    else:
        end_date = datetime.date.today()
 
    return quandl.get(query_list, 
            returns='pandas', 
            start_date=start_date,
            end_date=end_date,
            collapse='daily',
            order='asc'
            )
if __name__ == '__main__':
    apple_data = quandl_stocks('AAPL')


- Return all available data for stocks in TickerList from Quandl

In [284]:
data = quandl_stocks('FB')
data.columns = ['Ticker', 'Adj. Close']
data.Ticker = 'FB'
for stocks in TickerList[1:]: # iterate through all the other tickers, append to df
    newdata = quandl_stocks(stocks)
    if len(newdata) < 500:
        print(stocks,len(newdata)) # print tickers with missing/unavail info
    newdata.columns = ['Ticker', 'Adj. Close']
    newdata.Ticker = stocks
    data = pd.concat([data,newdata],axis=0)

LULU 0
HUBS 0
SAP 0
RHHBY 0
CDW 0
ADDYY 0
RDSA 0


In [ ]:
#datasaved = data.copy()

- Get rid of TRV and HLT data from Quandl, will replace with Yahoo data below

In [398]:
testdata = data[data.Ticker!="HLT"]
testdata = testdata[testdata.Ticker!="TRV"]

#### YAHOO finance data for the rest
- missing contains list to retrieve from yahoo


In [399]:
missing = ['LULU','HUBS','SAP','RHHBY','CDW','ADDYY','RDSA','TRV','HLT'] #TRV, HLT - add more data, quandl limited

- import CSVs and transform into same quandl formatting

In [400]:
missingdata = pd.read_csv('LULU.csv', index_col=0)
missingdata = missingdata[['Open','Adj Close']]
missingdata.columns = ['Ticker', 'Adj. Close']
missingdata.Ticker = "LULU"

for tickers in missing[1:]:
    newdata2 = pd.read_csv(tickers+'.csv', index_col=0)
    if len(newdata2) < 100:
        print(tickers,len(newdata2))
    newdata2 = newdata2[['Open','Adj Close']]
    newdata2.columns = ['Ticker', 'Adj. Close']
    newdata2.Ticker = tickers
    missingdata = pd.concat([missingdata,newdata2],axis=0)
    

In [343]:
#missingdatacopy = missingdata.copy()

In [401]:
missingdata.index = pd.to_datetime(missingdata.index)

- <b>StockPrices</b> below contains the final stocks data

In [403]:
StockPrices = pd.concat([testdata,missingdata],axis=0)

In [431]:
StockPrices.groupby(by='Ticker').size() #check the final output

Ticker
AAPL     2038
ACN      2039
ADBE     2038
ADDYY    1259
AVB      2039
BSX      2039
CDW      1164
COF      2039
CRM      2039
CSCO     2039
DAL      2039
DIS      2039
DRI      2039
EA       2039
ELLI     1715
EXR      2039
FB       1440
FORR     2039
GOOG      975
GWRE     1520
H        2039
HLT       782
HUBS      840
INTU     2039
JNJ      2039
LLY      2039
LULU     1259
LUV      2039
MMM      1985
MON      2039
MSFT     2039
NKE      2039
NSP      2039
NVDA     2039
OSK      2039
PCTY      981
PG       2039
RDSA     1259
RHHBY    1259
SAP      1259
SBUX     2039
SYK      2039
TMHC     1218
TMUS     2039
TRV      1259
UAL      2039
ULTI     2039
VMW      2039
Z         625
dtype: int64

In [445]:
StockPrices[(StockPrices.Ticker=="TRV") & (StockPrices.index == "2013-02-11")] #Retrieve obs at specific stock/datetime
StockPrices['Adj. Close'][(StockPrices.Ticker=="TRV") & (StockPrices.index == "2013-02-11")] #Retrieve price only

2013-02-11    70.599014
Name: Adj. Close, dtype: float64

In [493]:
ReviewsRaw.iloc[18028].DateRev[-4:]=='0000' #18028 date error, 23537

True

<p><a name="DC"></a></p>
### Data Cleaning

- Convert RawReviews review date to the same datetime format as stock prices

In [568]:
#removes 5 reviews with Jan/Aug 0, 0000 datetime, 1 2021, 2 4713's
ReviewsClean = ReviewsRaw[(ReviewsRaw.DateRev.str[-4:]!='0000') & (ReviewsRaw.DateRev.str[-4:]!='2021') & (ReviewsRaw.DateRev.str[-4:]!='4713')]
#remove 20 reviews with NaN review dates
ReviewsClean = ReviewsClean[ReviewsClean.DateRev.notnull()]
#one jan 0 datetime
ReviewsClean = ReviewsClean[ReviewsClean.DateRev.str[5:7]!='0,']
#drop the SAP concur data to align with stock data available
ReviewsClean = ReviewsClean[ReviewsClean.Name !=' SAP Concur ']

In [595]:
ReviewsClean.DateRev = pd.to_datetime(ReviewsClean.DateRev)

In [ ]:
numericfields = ['CareerOpp','CompBen','CultureVal','Rating','SnrMgmt','WorkLife']

In [607]:
ReviewsClean.CareerOpp = pd.to_numeric(ReviewsClean.CareerOpp)
ReviewsClean.CompBen = pd.to_numeric(ReviewsClean.CompBen)
ReviewsClean.CultureVal = pd.to_numeric(ReviewsClean.CultureVal)
ReviewsClean.Rating = pd.to_numeric(ReviewsClean.Rating)
ReviewsClean.SnrMgmt = pd.to_numeric(ReviewsClean.SnrMgmt)
ReviewsClean.WorkLife = pd.to_numeric(ReviewsClean.WorkLife)

In [672]:
#Reindex from 0 on the filtered clean data
ReviewsClean.index = range(0,len(ReviewsClean))

In [636]:
StockPrices.dtypes
#type(StockPrices.index)
ReviewsClean.dtypes

Advice                object
AuthLoc               object
AuthTitle             object
CareerOpp            float64
CompBen              float64
Cons                  object
CultureVal           float64
DateRev       datetime64[ns]
EmpType               object
Name                  object
OpCEO                 object
Outlook               object
Pros                  object
Rating               float64
Recommend             object
SnrMgmt              float64
UserSumm              object
WorkLife             float64
dtype: object

In [621]:
set(ReviewsClean.Recommend)

{'Recommends', nan, "Doesn't Recommend"}

<p><a name="DL"></a></p>
### Data Manipulation

- Manipulate cleaned, scraped data

In [632]:
#Join Overview Data with Review Level data by Company Name -- ['Ticker','Revenue','Industry','CompanyHQ']

RangeIndex(start=0, stop=150281, step=1)

In [677]:
ReviewsCleanMrg = pd.merge(ReviewsClean,OverviewPublic,left_on="Name",right_on="Name")

In [714]:
set(OverviewPublic.Name)-set(ReviewsCleanMrg.Name)
# Somehow Boston Scientific and Extra Space didn't make it into the scraped reviews.
# Also note that LinkedIn and Protiviti still in the reviews set but will get lost in the merge
#    because we dropped them from the table with available dataa

{' Boston Scientific ', ' Extra Space '}

In [711]:
set(ReviewsClean.Name) -set(OverviewPublic.Name)

{' LinkedIn ', ' Protiviti '}

In [725]:
pd.merge(ReviewsCleanMrg,StockPrices.reset_index(),left_on=["Ticker","DateRev"],right_on=["Ticker","index"])

,Advice,AuthLoc,AuthTitle,CareerOpp,CompBen,Cons,CultureVal,DateRev,EmpType,Name,...,WorkLife,CompanyHQ,Industry,Revenue,Size,Type,Public,Ticker,index,Adj. Close
0,Improve communication. Jobs need to have large...,"London, England (UK)",Former Employee - Allocator,2.0,2.5,can be very slow at at times except for christ...,NaN,2010-03-11,Former,Walt Disney Company,...,3.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2010-03-11,30.531949
1,Look to develop your employees and maximize th...,"Seattle, WA",Current Employee - Systems Administrator,3.0,3.0,Lack of options for promotion in many areas; H...,NaN,2010-03-11,Current,Walt Disney Company,...,5.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2010-03-11,30.531949
2,I would say that leadership needs to look at e...,NaN,Former Employee - Manager,2.0,1.5,"Poor wages, incompetent management, lack of ad...",NaN,2010-03-17,Former,Walt Disney Company,...,1.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2010-03-17,30.477766
3,increase focus on developing and maintaining c...,"Toronto, ON (Canada)",Current Employee - Licensing Coordinator (PT),1.5,2.0,Permanent Part-Time jobs do not include benefi...,NaN,2010-01-15,Current,Walt Disney Company,...,5.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2010-01-15,27.633175
4,Better communications and better organized env...,NaN,Current Employee - Anonymous Employee,3.0,2.0,"Has its fair share of bureaucracy, often has u...",NaN,2010-01-12,Current,Walt Disney Company,...,4.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2010-01-12,27.831845
5,NaN,Paris (France),Current Employee - Anonymous Employee,5.0,5.0,"Malgré la volonté innovante de l'entreprise, c...",5.0,2017-11-13,Current,Walt Disney Company,...,5.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2017-11-13,104.740000
6,.Train the managers better,"Hollywood, CA",Former Employee - Sales Associate,1.0,1.0,The pay is not as much as I was promised. Did ...,5.0,2017-11-13,Former,Walt Disney Company,...,4.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2017-11-13,104.740000
7,Strukturen müssten vereinfacht werden,"Munich, Bayern (Germany)",Current Employee - Manager,2.0,3.0,"US-zentriert statt global, Häufige Wechsel im ...",3.0,2017-11-20,Current,Walt Disney Company,...,2.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2017-11-20,102.750000
8,Be open to new ideas and refresh some top leve...,NaN,Former Employee - Anonymous Employee,4.0,4.0,"Still way behind in the fresh ideas, innovatio...",4.0,2017-11-20,Former,Walt Disney Company,...,3.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2017-11-20,102.750000
9,Be more transparent about decisions made that ...,NaN,Former Employee - Senior Manager,3.0,3.5,"For most departments, Disney is on the lower s...",NaN,2010-10-13,Former,Walt Disney Company,...,5.0,"Burbank, CA",Motion Picture Production & Distribution,$10+ billion (USD) per year,10000+ employees,Company - Public (DIS),True,DIS,2010-10-13,31.525299


,index,Ticker,Adj. Close
0,2012-05-18,FB,38.231800
1,2012-05-21,FB,34.030000
2,2012-05-22,FB,31.000000
3,2012-05-23,FB,32.000000
4,2012-05-24,FB,33.030000
5,2012-05-25,FB,31.910000
6,2012-05-29,FB,28.840000
7,2012-05-30,FB,28.190000
8,2012-05-31,FB,29.600000
9,2012-06-01,FB,27.720000
